In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

In [2]:
rf = 0.007
path1 = 'chart.csv'
path2 = 'SPY.csv'

In [3]:
def format_ameritrade_export(file_path):
    account = pd.read_csv(
        Path(file_path)
    )
    
    account.rename(columns = {"Account value": "value"}, inplace=True)
    account.dropna(inplace=True)
    account["Date"] = pd.to_datetime(account["Date"], format = '%m/%d/%Y')
    account.set_index("Date", inplace = True)
    #account["value"] = account["value"].str.replace(',', '')
    account["value"] = account["value"].astype(float, errors = 'raise')
    account.dropna(inplace=True)
    
    return account

In [4]:
def format_yfinance_export(file_path):
    spy_data = pd.read_csv(
        Path(file_path),
        index_col = 'Date',
        parse_dates = True,
        infer_datetime_format = True
    )
    
    spy_data.drop(["Open", "High", "Low", "Adj Close", "Volume"], axis=1, inplace=True)
    spy_data.dropna(inplace=True)
    
    return spy_data
    

In [5]:
def get_statistics(portfolio_data, spy_data, rf):
    statistics = ["daily_returns", "cumulative_returns", "average_annualized_returns", "volatility", "variance", "covariance", "annualized_sharpe_ratio"] 
    portfolio_statistics = [] 
    market_statistics = []
    rows = ["portfolio", "market"]
    
    daily_returns = portfolio_data["value"].pct_change()
    spy_daily_returns = spy_data["Close"].pct_change()
    portfolio_statistics.append(daily_returns)
    market_statistics.append(spy_daily_returns)
    
    cum_returns = (1+daily_returns).cumprod()
    spy_cum_returns = (1+spy_daily_returns).cumprod()
    portfolio_statistics.append(cum_returns)
    market_statistics.append(spy_cum_returns)
    
    average_annualized_returns = daily_returns.mean() * 252
    average_annualized_market_returns = spy_daily_returns.mean() * 252
    portfolio_statistics.append(average_annualized_returns)
    market_statistics.append(average_annualized_market_returns)
    
    volatility = portfolio_data["value"].std()
    market_volatility = spy_daily_returns.std()
    portfolio_statistics.append(volatility)
    market_statistics.append(market_volatility)
    
    variance = daily_returns.var()
    market_variance = spy_daily_returns.var()
    portfolio_statistics.append(variance)
    market_statistics.append(market_variance)
    
    covariance = daily_returns.cov(spy_daily_returns)
    portfolio_statistics.append(covariance)
    market_statistics.append(covariance)
    
    annualized_volatility = volatility * np.sqrt(252)
    annualized_market_volatility = market_volatility * np.sqrt(252)
    sharpe_ratio = average_annualized_returns - rf / annualized_volatility
    market_sharpe_ratio = average_annualized_market_returns - rf / annualized_market_volatility
    portfolio_statistics.append(sharpe_ratio)
    market_statistics.append(market_sharpe_ratio)
    
    main = [statistics, portfolio_statistics, market_statistics]
    
    return main

In [6]:
portfolio = format_ameritrade_export(path1)
market = format_yfinance_export(path2)
go = get_statistics(portfolio, market, rf)

In [10]:
display(go)

[['daily_returns',
  'cumulative_returns',
  'average_annualized_returns',
  'volatility',
  'variance',
  'covariance',
  'annualized_sharpe_ratio'],
 [Date
  2020-04-16             NaN
  2020-04-17    6.060606e-07
  2020-04-20    6.060602e-07
  2020-04-21    0.000000e+00
  2020-04-22    6.060599e-07
                    ...     
  2022-05-18   -2.078547e-02
  2022-05-19   -1.365933e-03
  2022-05-20   -5.790059e-03
  2022-05-23    3.173082e-02
  2022-05-24   -7.591020e-03
  Name: value, Length: 549, dtype: float64,
  Date
  2020-04-16         NaN
  2020-04-17    1.000001
  2020-04-20    1.000001
  2020-04-21    1.000001
  2020-04-22    1.000002
                  ...   
  2022-05-18    1.412288
  2022-05-19    1.410359
  2022-05-20    1.402193
  2022-05-23    1.446685
  2022-05-24    1.435704
  Name: value, Length: 549, dtype: float64,
  0.1775922201545729,
  2162.339266389591,
  8.878299099701469e-05,
  5.276273657791847e-05,
  0.17759201622794646],
 [Date
  2020-04-20         NaN
  20